In [1]:
import pandas as pd
pd.set_option('display.max_colwidth', 200)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 20)


from time import sleep
import requests

import geopandas as gpd

In [2]:
raw_data = "/Users/keenansmacbookairm3/Documents/GitHub/nyc_2024/raw_data/"

df = pd.read_csv(raw_data + "2020_official.csv", index_col=False, header=None)

df = df[[11, 12, 13, 20, 21]]

df.columns=['assembly_district', 'election_district', 'county', 'candidate', 'vote_count']
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79755 entries, 0 to 79754
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   assembly_district  79755 non-null  int64 
 1   election_district  79755 non-null  int64 
 2   county             79755 non-null  object
 3   candidate          79755 non-null  object
 4   vote_count         79755 non-null  object
dtypes: int64(2), object(3)
memory usage: 3.0+ MB


/var/folders/f2/xbsrplkj23z71l_fw6890_hr0000gn/T/ipykernel_73269/3067266419.py:3: DtypeWarning: Columns (21) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv(raw_data + "2020_official.csv", index_col=False, header=None)


In [3]:
df[['assembly_district', 'election_district']] = df[['assembly_district', 'election_district']].astype(str)

df['vote_count'] = df['vote_count'].astype(str)
df['vote_count'] = df['vote_count'].str.replace(',', '', regex=False).str.strip()

df['vote_count'] = pd.to_numeric(df['vote_count'], errors='coerce')

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 79755 entries, 0 to 79754
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   assembly_district  79755 non-null  object
 1   election_district  79755 non-null  object
 2   county             79755 non-null  object
 3   candidate          79755 non-null  object
 4   vote_count         79755 non-null  int64 
dtypes: int64(1), object(4)
memory usage: 3.0+ MB


In [4]:
df.groupby(['assembly_district', 'election_district', 'candidate'])['vote_count'].sum().unstack()

candidate                            Absentee / Military  Affidavit  \
assembly_district election_district                                   
23                1                                121.0        3.0   
                  10                               142.0       12.0   
                  11                               122.0       12.0   
                  12                               133.0       12.0   
                  13                               136.0       16.0   
...                                                  ...        ...   
87                95                                 0.0        0.0   
                  96                                46.0        5.0   
                  97                                86.0        3.0   
                  98                               115.0        4.0   
                  99                               118.0        4.0   

candidate                            Brock Pierce / Karla Ballard (Independence)  \
assembly_district election_district                                                
23                1                                                          0.0   
                  10                                                         1.0   
                  11                                                         1.0   
                  12                                                         1.0   
                  13                                                         2.0   
...                                                                          ...   
87                95                                                         NaN   
                  96                                                         0.0   
                  97                                                         0.0   
                  98                                                         1.0   
                  99                                                         0.0   

candidate                            Donald J. Trump / Michael R. Pence (Conservative)  \
assembly_district election_district                                                      
23                1                                                              141.0   
                  10                                                              44.0   
                  11                                                              38.0   
                  12                                                              26.0   
                  13                                                              32.0   
...                                                                                ...   
87                95                                                               NaN   
                  96                                                               2.0   
                  97                                                               4.0   
                  98                                                               8.0   
                  99                                                               5.0   

candidate                            Donald J. Trump / Michael R. Pence (Republican)  \
assembly_district election_district                                                    
23                1                                                            510.0   
                  10                                                           323.0   
                  11                                                           394.0   
                  12                                                           245.0   
                  13                                                           332.0   
...                                                                              ...   
87                95                                                             NaN   
                  96

In [5]:
def format_district(district_code):
    # Check the length and add zeros accordingly
    if len(district_code) == 1:
        return '00' + district_code  # Add two zeros for single digits
    elif len(district_code)== 2:
        return '0' + district_code   # Add one zero for two-digit numbers
    else:
        return district_code 

In [6]:
df['ed_code'] = df.assembly_district+df.election_district.apply(format_district)

df.head()

,assembly_district,election_district,county,candidate,vote_count,ed_code
0,65,1,New York,Public Counter,393,65001
1,65,1,New York,Manually Counted Emergency,0,65001
2,65,1,New York,Absentee / Military,263,65001
3,65,1,New York,Federal,7,65001
4,65,1,New York,Special Presidential,0,65001


In [7]:
df[(df.assembly_district =='82') & (df.election_district=='17')]

,assembly_district,election_district,county,candidate,vote_count,ed_code
23876,82,17,Bronx,Public Counter,504,82017
23877,82,17,Bronx,Manually Counted Emergency,0,82017
23878,82,17,Bronx,Absentee / Military,91,82017
23879,82,17,Bronx,Federal,1,82017
23880,82,17,Bronx,Special Presidential,0,82017
23881,82,17,Bronx,Affidavit,12,82017
23882,82,17,Bronx,Joseph R. Biden / Kamala D. Harris (Democratic),303,82017
23883,82,17,Bronx,Donald J. Trump / Michael R. Pence (Republican),195,82017
23884,82,17,Bronx,Donald J. Trump / Michael R. Pence (Conservative),22,82017
23885,82,17,Bronx,Joseph R. Biden / Kamala D. Harris (Working Families),21,82017


In [8]:
pivot_table = df.pivot(index='ed_code', columns='candidate', values='vote_count')

pivot_table = pivot_table.reset_index().set_index('ed_code')

pivot_table.head()

candidate,Absentee / Military,Affidavit,Brock Pierce / Karla Ballard (Independence),Donald J. Trump / Michael R. Pence (Conservative),Donald J. Trump / Michael R. Pence (Republican),Federal,Howie Hawkins / Angela Nicole Walker (Green),Jo Jorgensen / Jeremy Cohen (Libertarian),Joseph R. Biden / Kamala D. Harris (Democratic),Joseph R. Biden / Kamala D. Harris (Working Families),Manually Counted Emergency,Public Counter,Scattered,Special Presidential
ed_code,,,,,,,,,,,,,,
23001,121.0,3.0,0.0,141.0,510.0,0.0,5.0,10.0,216.0,27.0,0.0,797.0,6.0,0.0
23002,124.0,5.0,0.0,104.0,590.0,1.0,1.0,9.0,219.0,24.0,0.0,821.0,1.0,0.0
23003,96.0,5.0,2.0,80.0,556.0,4.0,8.0,10.0,199.0,16.0,0.0,772.0,2.0,0.0
23004,16.0,3.0,0.0,11.0,100.0,0.0,0.0,0.0,48.0,1.0,0.0,141.0,NaN,0.0
23005,187.0,18.0,0.0,41.0,445.0,3.0,5.0,8.0,286.0,23.0,0.0,606.0,2.0,0.0


In [9]:
pivot_table.info()

<class 'pandas.core.frame.DataFrame'>
Index: 5901 entries, 23001 to 87102
Data columns (total 14 columns):
 #   Column                                                 Non-Null Count  Dtype  
---  ------                                                 --------------  -----  
 0   Absentee / Military                                    5901 non-null   float64
 1   Affidavit                                              5901 non-null   float64
 2   Brock Pierce / Karla Ballard (Independence)            5760 non-null   float64
 3   Donald J. Trump / Michael R. Pence (Conservative)      5760 non-null   float64
 4   Donald J. Trump / Michael R. Pence (Republican)        5760 non-null   float64
 5   Federal                                                5901 non-null   float64
 6   Howie Hawkins / Angela Nicole Walker (Green)           5760 non-null   float64
 7   Jo Jorgensen / Jeremy Cohen (Libertarian)              5760 non-null   float64
 8   Joseph R. Biden / Kamala D. Harris (Democratic) 

In [10]:
pivot_table['precinct_total'] = pivot_table.iloc[:, list(range(2, 5)) + list(range(6, 10))].sum(axis=1)

pivot_table['trump_total'] = pivot_table.iloc[:,3:5].sum(axis=1)
pivot_table['trump_share'] = pivot_table.trump_total/ pivot_table.precinct_total

pivot_table['biden_total'] = pivot_table.iloc[:,8:10].sum(axis=1)
pivot_table['biden_share'] = pivot_table.biden_total/ pivot_table.precinct_total


pivot_table = pivot_table.iloc[:, -5:].reset_index()
pivot_table.head()

candidate,ed_code,precinct_total,trump_total,trump_share,biden_total,biden_share
0,23001,909.0,651.0,0.716172,243.0,0.267327
1,23002,947.0,694.0,0.732841,243.0,0.256600
2,23003,871.0,636.0,0.730195,215.0,0.246843
3,23004,160.0,111.0,0.693750,49.0,0.306250
4,23005,808.0,486.0,0.601485,309.0,0.382426


In [11]:
pivot_table.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5901 entries, 0 to 5900
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ed_code         5901 non-null   object 
 1   precinct_total  5901 non-null   float64
 2   trump_total     5901 non-null   float64
 3   trump_share     5631 non-null   float64
 4   biden_total     5901 non-null   float64
 5   biden_share     5631 non-null   float64
dtypes: float64(5), object(1)
memory usage: 276.7+ KB


In [12]:
geo_file = 'NYS_Elections_Districts_and_Polling_Locations_-3344009424923235749.geojson'

gdf = gpd.read_file('/Users/keenansmacbookairm3/Downloads/'+ geo_file)

gdf.columns = gdf.columns.str.lower()
gdf = gdf[gdf.municipality.str.contains('New York City')]

gdf.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
Index: 4345 entries, 8955 to 13299
Data columns (total 5 columns):
 #   Column             Non-Null Count  Dtype   
---  ------             --------------  -----   
 0   objectid           4345 non-null   int32   
 1   county             4345 non-null   object  
 2   municipality       4345 non-null   object  
 3   election_district  4345 non-null   object  
 4   geometry           4345 non-null   geometry
dtypes: geometry(1), int32(1), object(3)
memory usage: 186.7+ KB


In [13]:
new_gdf = pivot_table.merge(gdf[['county', 'geometry', 'election_district']], 
                      left_on='ed_code', 
                      right_on= 'election_district', 
                      how='right',
                      suffixes=('_result', '_map')
                      ).drop(columns='ed_code')


In [14]:
new_gdf

,precinct_total,trump_total,trump_share,biden_total,biden_share,county,geometry,election_district
0,909.0,651.0,0.716172,243.0,0.267327,Queens,"POLYGON ((-73.92033 40.56223, -73.92078 40.56148, -73.91989 40.56168, -73.91986 40.56148, -73.91982 40.56127, -73.91993 40.56126, -73.92004 40.56125, -73.92014 40.56123, -73.92013 40.561, -73.9201...",23001
1,947.0,694.0,0.732841,243.0,0.256600,Queens,"POLYGON ((-73.91018 40.56493, -73.91039 40.56452, -73.91037 40.56441, -73.91055 40.56413, -73.9106 40.56403, -73.91065 40.56393, -73.91069 40.56384, -73.9107 40.56342, -73.91027 40.5624, -73.90958...",23002
2,871.0,636.0,0.730195,215.0,0.246843,Queens,"POLYGON ((-73.86287 40.56684, -73.86268 40.56653, -73.86268 40.56652, -73.86269 40.56651, -73.86269 40.56651, -73.86271 40.56651, -73.86272 40.56651, -73.86277 40.56658, -73.86296 40.56682, -73.86...",23003
3,160.0,111.0,0.693750,49.0,0.306250,Queens,"POLYGON ((-73.85274 40.57061, -73.85312 40.57051, -73.85332 40.57044, -73.85358 40.57035, -73.85381 40.57034, -73.85385 40.57034, -73.85397 40.57032, -73.8541 40.57026, -73.85423 40.57015, -73.854...",23004
4,808.0,486.0,0.601485,309.0,0.382426,Queens,"POLYGON ((-73.84878 40.57206, -73.84887 40.57202, -73.84897 40.57199, -73.84902 40.57196, -73.84907 40.57195, -73.84914 40.57192, -73.84925 40.57186, -73.84936 40.57183, -73.84941 40.5718, -73.849...",23005
...,...,...,...,...,...,...,...,...
4340,689.0,516.0,0.748911,171.0,0.248186,Richmond,"POLYGON ((-74.17092 40.53924, -74.1716 40.53866, -74.17196 40.53837, -74.17227 40.5381, -74.17321 40.53801, -74.1734 40.538, -74.1734 40.538, -74.1734 40.538, -74.17359 40.53799, -74.17359 40.5379...",62019
4341,392.0,90.0,0.229592,301.0,0.767857,Queens,"POLYGON ((-73.75598 40.59779, -73.75606 40.59778, -73.75614 40.59777, -73.75621 40.59775, -73.75629 40.59773, -73.75635 40.5977, -73.75642 40.59767, -73.7566 40.59758, -73.75678 40.59749, -73.7568...",31001
4342,217.0,45.0,0.207373,171.0,0.788018,Queens,"POLYGON ((-73.81788 40.67385, -73.81697 40.67368, -73.81606 40.67352, -73.81516 40.67335, -73.81572 40.67154, -73.81628 40.66972, -73.81685 40.66791, -73.81741 40.6661, -73.81833 40.66625, -73.819...",31021
4343,307.0,27.0,0.087948,280.0,0.912052,Bronx,"POLYGON ((-73.89287 40.84346, -73.89167 40.84326, -73.89152 40.84324, -73.89128 40.84315, -73.89038 40.84279, -73.88986 40.84253, -73.88933 40.84226, -73.88882 40.84198, -73.88831 40.84169, -73.88...",79022


In [15]:
new_gdf = new_gdf[(new_gdf.biden_total > 0) & (new_gdf.trump_total > 0)]

new_gdf = gpd.GeoDataFrame(new_gdf)

new_gdf.head()

,precinct_total,trump_total,trump_share,biden_total,biden_share,county,geometry,election_district
0,909.0,651.0,0.716172,243.0,0.267327,Queens,"POLYGON ((-73.92033 40.56223, -73.92078 40.56148, -73.91989 40.56168, -73.91986 40.56148, -73.91982 40.56127, -73.91993 40.56126, -73.92004 40.56125, -73.92014 40.56123, -73.92013 40.561, -73.9201...",23001
1,947.0,694.0,0.732841,243.0,0.256600,Queens,"POLYGON ((-73.91018 40.56493, -73.91039 40.56452, -73.91037 40.56441, -73.91055 40.56413, -73.9106 40.56403, -73.91065 40.56393, -73.91069 40.56384, -73.9107 40.56342, -73.91027 40.5624, -73.90958...",23002
2,871.0,636.0,0.730195,215.0,0.246843,Queens,"POLYGON ((-73.86287 40.56684, -73.86268 40.56653, -73.86268 40.56652, -73.86269 40.56651, -73.86269 40.56651, -73.86271 40.56651, -73.86272 40.56651, -73.86277 40.56658, -73.86296 40.56682, -73.86...",23003
3,160.0,111.0,0.693750,49.0,0.306250,Queens,"POLYGON ((-73.85274 40.57061, -73.85312 40.57051, -73.85332 40.57044, -73.85358 40.57035, -73.85381 40.57034, -73.85385 40.57034, -73.85397 40.57032, -73.8541 40.57026, -73.85423 40.57015, -73.854...",23004
4,808.0,486.0,0.601485,309.0,0.382426,Queens,"POLYGON ((-73.84878 40.57206, -73.84887 40.57202, -73.84897 40.57199, -73.84902 40.57196, -73.84907 40.57195, -73.84914 40.57192, -73.84925 40.57186, -73.84936 40.57183, -73.84941 40.5718, -73.849...",23005


In [17]:
type(new_gdf)

geopandas.geodataframe.GeoDataFrame

In [18]:
file_name = '2020_official_geojson.geojson'


new_gdf.to_file("/Users/keenansmacbookairm3/Documents/GitHub/nyc_2024/semi_processed_data/"+file_name, driver="GEOJSON")